In [1]:
import os

In [2]:
%pwd

'c:\\Users\\SIR\\Credit-Card-Default-Prediction-Project-Pwskills\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\SIR\\Credit-Card-Default-Prediction-Project-Pwskills'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    model_file: Path
    evaluation_metrics_file: Path
    test_file: Path

In [7]:
from ccdp.constants import *
from ccdp.utils.common import read_yaml, create_directories

In [12]:
#from src.ccdp.entity.data_evaluation import ModelEvaluationConfig

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    # Model Evaluation Configuration
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        evaluation_metrics_path = Path(config.evaluation_metrics_file)
        create_directories([evaluation_metrics_path.parent])

        model_evaluation_config = ModelEvaluationConfig(
            model_file=Path(config.model_file),
            evaluation_metrics_file=evaluation_metrics_path,
            test_file=Path(config.test_file)
        )

        return model_evaluation_config

In [13]:
import pandas as pd
import pickle
from sklearn.metrics import accuracy_score, classification_report
from pathlib import Path
from ccdp.logging import logger

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def load_model(self):
        with open(self.config.model_file, "rb") as model_file:
            self.model = pickle.load(model_file)
        logger.info(f"Model loaded from {self.config.model_file}")

    def evaluate(self):
        # Load test dataset
        test_df = pd.read_csv(self.config.test_file)
        X_test = test_df.drop(columns=['default'])  # Assuming 'default' is the target column
        y_test = test_df['default']
        
        # Make predictions
        y_pred = self.model.predict(X_test)
        
        # Evaluate the model
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred)
        
        # Save metrics
        self.save_metrics(accuracy, report)
        
        logger.info(f"Model evaluation completed with accuracy: {accuracy}")

    def save_metrics(self, accuracy, report):
        with open(self.config.evaluation_metrics_file, "w") as metrics_file:
            metrics_file.write(f"Accuracy: {accuracy}\n\n")
            metrics_file.write("Classification Report:\n")
            metrics_file.write(report)
        logger.info(f"Evaluation metrics saved to {self.config.evaluation_metrics_file}")


In [14]:
#from src.ccdp.config.configuration import ConfigurationManager
#from src.ccdp.components.model_evaluation import ModelEvaluation

try:
    # Configuration for Model Evaluation
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    
    # Initialize the ModelEvaluation component
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    
    # Load the model and evaluate
    model_evaluation.load_model()
    model_evaluation.evaluate()
    
    logger.info("Model evaluation pipeline completed successfully.")

except Exception as e:
    logger.error(f"Pipeline failed due to: {e}")


[2024-08-21 00:18:21,732: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-21 00:18:21,737: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-21 00:18:21,740: INFO: common: created directory at: artifacts]
[2024-08-21 00:18:21,745: INFO: common: created directory at: artifacts\model_evaluation]
[2024-08-21 00:18:22,077: INFO: 3709180523: Model loaded from artifacts\model_training\models\model.pkl]
[2024-08-21 00:18:22,296: INFO: 3709180523: Evaluation metrics saved to artifacts\model_evaluation\metrics.txt]
[2024-08-21 00:18:22,298: INFO: 3709180523: Model evaluation completed with accuracy: 0.818]
[2024-08-21 00:18:22,302: INFO: 3051264557: Model evaluation pipeline completed successfully.]
